# Routing algorithm

This notebook solves a vehicle routing / scheduling problem with **time-dependent travel times**.

It is organized as:
1. **Setup & parameters**
2. **Data loading**
3. **Distance / travel-time matrices** (shortest paths on a graph)
4. **MILP model** (DOcplex)
5. **Post-processing & plots**

> Tip: Keep the input data in `data/` 


In [ ]:
# Reproducibility knobs
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
import os
import pickle
import time
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt
import pandas as pd
import googlemaps
from copy import deepcopy
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from docplex.mp.model import Model
np.set_printoptions(precision=6, suppress=True)

# Project paths
ROOT = Path('.')
DATA_DIR = ROOT / 'data'

In [ ]:
# --- User inputs: ---
# List of coordinates ("lat,lng") including depot as the first element
coords_list = [
    "60.,24.",  # Depot
    "60.,24.",  # BS1
    "60.,24.",  # BS2
    "60.,24.",  # BS3
    "60.,24.",  # BS4
    "60.,24.",  # BS5
    "60.,24.",  # BS6
    "60.,24.",  # BS7
    "60.,24.",  # BS8
    "60.,24.",  # BS9
    "60.,24.",  # BS10
]

## Step 1. Data Input and Preprocessing

In [ ]:
depo_index = 0              # index of the depot in coords_list

# Define vehicle operating cost (Euros per km) and value of time (Euros per hour).
C_o = 2.5502  # Euros per km for truck 1.2751
C_T = 28.7   # Euros per hour 14.35

# Define additional parameters.
Truck_capacity = 6         # Vehicle capacity in packages
Cus_demand = 1             # Demand per customer (in packages)
MAX_TRAVEL_DISTANCE = 100  # Maximum allowed travel distance in kilometers
service_time = 25/60         # Service time at each customer in hour

# For cumulative time constraints, set a large upper bound.
B_bar = 3            # Maximum allowed cumulative travel time (hour)
D_bar = MAX_TRAVEL_DISTANCE  # Maximum allowed cumulative travel distance (meters)
# Big-M 
M_big = B_bar + 10  # something larger than any possible total route time

interval = 1.0    # 1 hour
H = 2*B_bar       # 2 + of time

# pick your start‐hour anywhere in 0…23
start_hour = 13

EV_cost = 50
MC_mean = 0.6
MC_std = 0.1
MC = 10000
 
# lb_factor = 1          # R  # This should be 1 if MILP_filename is "mean"

MILP_filename = "lb_2024-11-20.csv"       # Mean_Out
True_filename = "True_2024-11-20.csv"

### Data visualization

In [ ]:
# Load the files
df_MILP = pd.read_csv(DATA_DIR / "Mean_Out_2024-11-20.csv")
df_true = pd.read_csv(DATA_DIR / "True_2024-11-20.csv")

# df_MILP.loc[:, df_MILP.columns != "Hours"] *= lb_factor

# Reshape both datasets into long format
df_MILP_melted = df_MILP.melt(id_vars="Hours", var_name="ID", value_name="Mean")
df_true_melted = df_true.melt(id_vars="Hours", var_name="ID", value_name="True")

# Merge on Hours and ID
df_merged = pd.merge(df_MILP_melted, df_true_melted, on=["Hours", "ID"])

# Convert ID to string for better legend readability
df_merged["ID"] = df_merged["ID"].astype(str)

# Plot
plt.figure(figsize=(12, 7))

ids = df_merged["ID"].unique()
colors = plt.cm.tab20.colors

for i, id_val in enumerate(ids):
    subset = df_merged[df_merged["ID"] == id_val]
    color = colors[i % len(colors)]
    plt.plot(subset["Hours"], subset["True"], linestyle="-", marker="o", markersize=3,
             color=color, label=f"True {id_val}")
    plt.plot(subset["Hours"], subset["Mean"], linestyle="--", marker="o", markersize=3,
             color=color, label=f"Mean {id_val}")

plt.xlabel("Hours")
plt.ylabel("Values")
plt.title("Mean vs True Speeds per ID with markers")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize="small", ncol=2)
plt.tight_layout()
plt.show()

## Step 2. Build Distance and Travel Time Matrices

In [ ]:
# # Initialize Google Maps client

load_dotenv()
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAPS_API_KEY"))

# # Number of locations
# epoch_count = len(coords_list)

# # Preallocate matrices
# distance_matrix = np.zeros((epoch_count, epoch_count), dtype=float)
# travel_time_matrix = np.zeros((epoch_count, epoch_count), dtype=float)

# # Build directed graph
# G = nx.DiGraph()
# for idx, coord in enumerate(coords_list):
#     G.add_node(idx, coord=coord)
    
    
# # Batch API calls: one request per origin, querying all destinations
# def fetch_matrix():
#     for i, origin in enumerate(coords_list):
#         result = gmaps.distance_matrix(
#             origins=[origin],
#             destinations=coords_list,
#             mode='driving',
#             departure_time=datetime.now()
#         )
#         elements = result['rows'][0]['elements']
#         for j, elem in enumerate(elements):
#             if elem['status'] == 'OK':
#                 dist = elem['distance']['value']         # meters
#                 tt   = elem['duration_in_traffic']['value']  # seconds
#             else:
#                 dist = float('inf')
#                 tt   = float('inf')

#             # store
#             distance_matrix[i, j] = dist
#             travel_time_matrix[i, j] = tt

#             # update graph edge
#             G.add_edge(i, j, distance=dist, time=tt)

# fetch_matrix()

# # travel_time_min = travel_time_matrix / 60.0
# # travel_time_min = np.round(travel_time_min, 1)
# # print(travel_time_min)

# # Execute fetch
# # epoch = datetime.now()
# # print(f"Matrix fetched at {epoch.isoformat()}")
# # np.save("distance_matrix.npy", distance_matrix)

# max_speed_matrix = np.full((num_locations, num_locations), 80.0)  # mostly 80 km/h
# # override the known roads:
# max_speed_matrix[3, 5] = max_speed_matrix[5, 3] = max_speed_matrix[5, 8] = max_speed_matrix[8, 5] = max_speed_matrix[8, 9] = max_speed_matrix[9, 8] = max_speed_matrix[9, 10] = max_speed_matrix[10, 9] = 100.0   # motorway
# max_speed_matrix[5, 6] = max_speed_matrix[6, 5] = max_speed_matrix[6, 7] = max_speed_matrix[7, 6] = max_speed_matrix[7, 9] = max_speed_matrix[9, 7] = 30.0    # local street
# max_speed_matrix[8, 6] = max_speed_matrix[6, 8] = 50.0 # local street
# max_speed_matrix[0, 9] = max_speed_matrix[9, 0] = 60.0 # local street
# max_speed_matrix[0, 10] = max_speed_matrix[10, 0] = 70.0 # local street

# speed_matrix_list = []
# travel_time_matrix_list = []
# full_cost_list = []
# for t in range(5):
#     # 1) sample a random factor in [0.2, 1.0] for each edge
#     rand_factor = np.random.uniform(0.5, 1.0, size=(num_locations, num_locations))
    
#     # 2) enforce symmetry so speed[i,j] == speed[j,i]:
#     rand_factor = np.triu(rand_factor) + np.triu(rand_factor, 1).T
    
#     # 3) apply to the original matrix
#     speed_matrix_list.append(max_speed_matrix * rand_factor)
#     travel_time_matrix_list.append((distance_matrix / 1000) / (speed_matrix_list[t]))
    
#     full_cost_list.append(((distance_matrix / 1000) / (speed_matrix_list[t])) * C_T)


# travel_time_min = travel_time_matrix_list[0] * 60.0
# travel_time_min = np.round(travel_time_min, 1)
# print(travel_time_min)

### Use saved Graph and Distance matrix

In [ ]:
with open(DATA_DIR / "graph_G.pkl", "rb") as f:
    G = pickle.load(f)

# Distance matrix for fully connected network    
distance_matrixO = np.load(DATA_DIR / "distance_matrix.npy")
# print(distance_matrix/1000)

distance_matrix = distance_matrixO * 2   # scaling factor to make the problem more challenging (longer distances, more time, more differentiation between routes)

# number of nodes
num_locations = distance_matrix.shape[0]
N = num_locations

# 1) Read the CSV, using the 'Hours' column as the index
dfO = pd.read_csv(DATA_DIR/MILP_filename, index_col="Hours")
# 2) Ensure index and column types are integers
dfO.index = dfO.index.astype(int)
dfO.index.name = "Hour"
dfO.columns = dfO.columns.astype(int)
# print(df.head())

df = dfO  #* lb_factor

# build a wrapped list of the 24 hours starting from start_hour
####################################################################################################
Pred_hours = [(start_hour + hh) % 24 for hh in range(len(df.index))]    # time-dependent
# Pred_hours = [8] * len(df.index)                              # time-independent
####################################################################################################

# To build a connected network (not fully connected tho)
# dynamic_edges[(i,j)] = RoadID
dynamic_edges = {
    (0,1): 165, (0,4):147, (0,7):164, (0,9):151, (0,10):116,
    (1,2):128, (2,3):160, (2,4):131, (3,5):109, (4,5):147,
    (5,8):154, (7,8):164, (8,9):117, (9,10):117,
}

# static_edges[(i,j)] = constant weight
static_edges = {
    (5,6): 30,
    (6,7): 30,
    (6,8): 50,
    (7,9): 30,
}

# prepare container
speed_matrix_list = []

for h in Pred_hours:               # loops 0 through 23
    M = np.full((N, N), 0, dtype=int)
    Dis = np.full((N,N), np.inf)   # (distance will not change for each h)

    speeds = df.loc[h]           # a Series indexed by RoadID (speed will chagne at each h)

    # dynamic edges
    for (i,j), Rid in dynamic_edges.items():
        w = speeds[Rid]
        M[i, j] = w  
        M[j, i] = w
        
        Dis[i, j] = distance_matrix[i, j]     # bi-directional distances
        Dis[j, i] = distance_matrix[j, i]

    # static edges
    for (i,j), w in static_edges.items():
        M[i, j] = w
        M[j, i] = w
        
        Dis[i, j] = distance_matrix[i, j]     # bi-directional distances
        Dis[j, i] = distance_matrix[j, i]

    speed_matrix_list.append(M)
    
    
# Convert your distance to kilometers
distance_km = Dis / 1000

# Build your list of 24 travel-time matrices (in hours)
travel_time_matrix_list = []
for h, speed_mat in enumerate(speed_matrix_list):
    # time (h) = distance (km) / speed (km/h)
    safe_speed = np.where(speed_mat > 0, speed_mat, 1e-6)
    tt = distance_km / safe_speed
    np.fill_diagonal(tt, 0)
    
    travel_time_matrix_list.append(tt)
    
travel_time_min = travel_time_matrix_list[0] * 60.0
travel_time_min = np.round(travel_time_min, 1)
print(travel_time_min)

## Step 3. Dijkstra's Algorithm for Shortest Paths

In [ ]:
num_intervals    = len(travel_time_matrix_list)   # considered time frames
distance_cost = (Dis / 1000) * C_o   # Euros

# prepare containers
full_cost_list        = []    # list of cost matrices
updated_cost_list      = []    # list of (N×N) cost matrices
updated_distance_list     = []    # list of (N×N) base‐distance matrices
updated_travel_time_list = []
shortest_paths_list     = []    # list of dicts mapping (i,j)->path

# for each interval (even tho the Distances are the same, for generic purpose/for different W)
for t in range(num_intervals):
    # --- 1) build full‐weight for this time t ---
    tt_matrix = travel_time_matrix_list[0]  # in seconds
    time_cost = tt_matrix * C_T    # Euros
    W = distance_cost          # Euros
    
    full_cost_list.append(W)
    
    # --- 2) run bidir‐Dijkstra on W ---
    updated_full_cost  = np.zeros((num_locations, num_locations))
    updated_base_dist  = np.zeros((num_locations, num_locations))
    updated_time_dist  = np.zeros((num_locations, num_locations))
    spaths             = {}
    
    # local weight function that closes over W
    def weight_cost(u, v, data):
        return W[u, v]
    
    for i in range(num_locations):
        for j in range(num_locations):
            if i == j:
                updated_full_cost[i, j] = 0.0
                updated_base_dist[i, j] = 0.0
                updated_time_dist[i, j] = 0.0
                spaths[(i, j)]         = [i]
            else:
                cost, path = nx.bidirectional_dijkstra(G, i, j, weight=weight_cost)
                updated_full_cost[i, j] = cost
                # sum raw distances (meters) along path
                total_d = sum(Dis[u, v] for u, v in zip(path[:-1], path[1:]))
                total_time = sum(travel_time_matrix_list[t][u, v] for u, v in zip(path[:-1], path[1:]))
                
                updated_base_dist[i, j] = total_d
                updated_time_dist[i, j] = total_time
                spaths[(i, j)] = path
    
    # store for this t
    updated_cost_list.append(updated_full_cost)
    updated_distance_list.append(updated_base_dist)
    updated_travel_time_list.append(updated_time_dist)
    shortest_paths_list.append(spaths)
    
distance_matrix_ = updated_distance_list[0]
travel_time_matrix_list = updated_travel_time_list

## Step 4. Formulate and Solve the Optimization Model

### Define Parameters

In [ ]:
start = time.time()

# Define Cost Matrices and Other Parameters

# Compute cost matrices.
distance_cost = (distance_matrix / 1000) * C_o  # Euros
time_cost_matrices = [tt * C_T for tt in travel_time_matrix_list]
full_cost_matrices = time_cost_matrices  # Combined cost (in Euros)

# Create dictionaries for model parameters.
V_range = list(range(num_locations))       # Nodes 0,...,n (0 = depot)
V_customers = list(range(1, num_locations))  # Customer nodes

# Set of arcs (directed edges) for i != j.
E = [(i, j) for i in V_range for j in V_range if i != j]

# Convert matrices into dictionaries (keys: (i,j)).
d_dict = {(i, j): distance_matrix[i, j]/1000 for (i, j) in E}
t_dict = {(i, j, h): travel_time_matrix_list[h][i, j] for (i, j) in E for h in range(H)}   # for each (i,j) and h, store the travel time in hours
cost_dict = {(i, j, h): full_cost_matrices[h][i, j]for (i, j) in E for h in range(H)}

# Service time: 0 for depot, fixed service time for customers.
s_dict = {i: (0 if i == 0 else service_time) for i in V_range}

# Demand dictionary: only for customers.
N_dict = {i: Cus_demand for i in V_customers}

### Optimization Problem

In [ ]:
mdl = Model("VRP_with_FullCost")

# Decision variables:
# x[i,j] = 1 if arc (i,j) is used; 0 otherwise.
x = mdl.binary_var_dict(E, name="x")

# z[i,j,h] = 1 if arc (i,j) is used and departure from i falls in interval h
z = mdl.binary_var_dict(((i, j, h) for (i,j) in E for h in range(H)), name="z")

# l[i] = departure time from node i (hours after time=0)
l = mdl.continuous_var_dict(V_range, lb=0, name="l")

# y[i,j] = number of packages carried along arc (i,j).
y = mdl.continuous_var_dict(E, lb=0, name="y")

# T[i,j] = cumulative travel time upon arriving at j via arc (i,j).
T = mdl.continuous_var_dict(E, lb=0, name="T")

# D[i,j] = cumulative travel distance upon arriving at j via arc (i,j).
D_var = mdl.continuous_var_dict(E, lb=0, name="D")

# k = number of vehicles used.
k = mdl.integer_var(lb=0, name="k")

# Objective: Minimize the total full cost over all used arcs.
mdl.minimize(mdl.sum(cost_dict[i,j,h] * z[i,j,h] for (i,j) in E for h in range(H))+ (num_locations-1)*(service_time)*C_T + EV_cost * k)


# --- Constraints ---

# 1. Each customer must be visited exactly once. Vehicles must leave from and return to the depot.
# C1
for j in V_customers:
    mdl.add_constraint(mdl.sum(x[i, j] for i in V_range if i != j) == 1, ctname=f"visit_in_{j}")
for i in V_customers:
    mdl.add_constraint(mdl.sum(x[i, j] for j in V_range if i != j) == 1, ctname=f"visit_out_{i}")
# C2  
mdl.add_constraint(mdl.sum(x[0, i] for i in V_customers) == k, ctname="depot_depart")
mdl.add_constraint(mdl.sum(x[i, 0] for i in V_customers) == k, ctname="depot_return")

# 2. Capacity constraints on package flow.
# C3
for (i, j) in E:
    mdl.add_constraint(y[i, j] <= Truck_capacity * x[i, j], ctname=f"capacity_{i}_{j}")
# Flow conservation for package delivery at each customer.
# C4
for i in V_customers:
    mdl.add_constraint(
        mdl.sum(y[j, i] for j in V_range if j != i) - mdl.sum(y[i, j] for j in V_range if j != i) == N_dict[i],
        ctname=f"demand_{i}"
    )
    
# 3. Time interval selection.
# C5
for (i, j) in E:
    mdl.add_constraint(mdl.sum(z[i, j, h] for h in range(H)) == x[i, j], ctname=f"pick_interval_{i}_{j}")
    
# 4. Cumulative travel time constraints.
# (a) Time flow balance for each nodes.
for i in V_range:
    for j in V_customers:
        if i != j:
            # only enforce when you actually use (i,j), if x[i,j]=1, then T[i,j] = d[i] + travel_ij
            # C6
            mdl.add_constraint(T[i,j] >= l[i] + mdl.sum(t_dict[i,j,h] * z[i,j,h] for h in range(H)) - M_big*(1 - x[i,j]),ctname=f"Tdef_lo_{i}_{j}")
            # C7
            mdl.add_constraint(T[i,j] <= l[i] + mdl.sum(t_dict[i,j,h] * z[i,j,h] for h in range(H)) + M_big*(1 - x[i,j]),ctname=f"Tdef_hi_{i}_{j}")  
    
# (b) Upper bound on travel time along paths arriving at a customer.
# for i in V_range:
#     for j in V_customers:
#         if i != j:
#             mdl.add_constraint(T[i, j] <= B_bar * x[i, j], ctname=f"time_ub_{i}_{j}")
# C8
for i in V_customers:
    mdl.add_constraint(l[i] <= B_bar, ctname=f"time_ub_{i}")

mdl.add_constraint(l[0] == 0, ctname="start_at_zero")

# (c) Flow of depurture time, leaving node i, l[i]
# T[j,i] = arrival at i via (j,i) so departure d[i] must be T[j,i] + s_i on the one used arc
for i in V_customers:
    for j in V_range:
        if i != j:
            # if x[j,i]=1 then these two must coincide, otherwise the big‐M turns it off
            # C9
            mdl.add_constraint( l[i] >= T[j,i] + s_dict[i]   - M_big*(1 - x[j,i]), ctname=f"dep_lb_{j}_{i}")
            # C10
            mdl.add_constraint( l[i] <= T[j,i] + s_dict[i]   + M_big*(1 - x[j,i]), ctname=f"dep_ub_{j}_{i}")

# (d) force z to match departure interval, h
# C11
for (i,j) in E:
  for h in range(H):
      mdl.add_constraint(l[i] >=  h*interval   - M_big*(1 - z[i,j,h]), ctname=f"bin_lo_{i}_{j}_{h}")
      mdl.add_constraint(l[i] <= (h+1)*interval + M_big*(1 - z[i,j,h]), ctname=f"bin_hi_{i}_{j}_{h}")  
    

# 5. Cumulative travel distance constraints.
# (a) Distance flow balance for each customer.
# C12
for i in V_customers:
    mdl.add_constraint(
        mdl.sum(D_var[i, j] for j in V_range if i != j) - mdl.sum(D_var[j, i] for j in V_range if j != i) ==
        mdl.sum(d_dict[i, j] * x[i, j] for j in V_range if i != j),
        ctname=f"distance_flow_{i}"
    )
# (b) Upper bound on cumulative distance along arcs arriving at a customer.
# C13
for i in V_range:
    for j in V_customers:
        if i != j:
            mdl.add_constraint(D_var[i, j] <= (D_bar - d_dict[j, 0]) * x[i, j], ctname=f"dist_ub_{i}_{j}")
# (c) Lower bound on cumulative distance for arcs leaving a customer.
# C14
for i in V_customers:
    for j in V_range:
        if i != j:
            mdl.add_constraint(D_var[i, j] >= (d_dict[i, j] + d_dict[0, i]) * x[i, j], ctname=f"dist_lb_{i}_{j}")
# (d) For arcs returning to the depot, ensure total distance does not exceed D_bar.
# C15
for i in V_customers:
    mdl.add_constraint(D_var[i, 0] <= D_bar * x[i, 0], ctname=f"return_dist_{i}")
# (e) For arcs departing the depot, set the distance equal to the distance from the depot.
for i in V_customers:
    mdl.add_constraint(D_var[0,i] == d_dict[0, i] * x[0, i], ctname=f"departure_dist_{i}")

### Solve the Problem

In [ ]:
solution = mdl.solve(log_output=True)

if solution:
    mdl.print_solution()
else:
    print("No solution found.")

### Extract Optimal Solutions

In [ ]:
EVs = k.solution_value

# Assume depot is node 0 and V_range is the list of all nodes.
routes = []
depot = 0

# Iterate over all arcs leaving the depot to start a route.
for j in V_range:
    if j == depot:
        continue
    # Check if the arc from depot to j is used.
    if solution.get_value(x[depot, j]) > 0.5:
        route = [depot, j]
        current = j
        # Continue following the route until we return to the depot.
        while current != depot:
            next_node = None
            # Look for the arc leaving current with value 1.
            for k in V_range:
                if k != current and solution.get_value(x[current, k]) > 0.5:
                    next_node = k
                    break
            # If no outgoing arc is found, break (shouldn't happen in a complete route).
            if next_node is None:
                break
            route.append(next_node)
            current = next_node
        routes.append(route)

# Print the routes.
print("Routes:")
for r in routes:
    print(" -> ".join(map(str, r)))
    
if solution:
    print(f" → Optimal total cost: €{mdl.objective_value:.2f}")
    
end = time.time()

print("Runtime:", end - start, "seconds")

## Step 5. Performance Evaluations
Uses: routes (list of lists), Dis (meters), service_time (hours),
start_hour (int), interval (hours), dynamic_edges, static_edges,
V_range, V_customers, l (docplex vars), solution (solve result)

### 1) Extract MILP departure times l[i]

In [ ]:
l_MILP = {i: float(solution.get_value(l[i])) for i in V_range}

# Load modified speeds and rebuild speed matrices for Pred_hours
df_mod = pd.read_csv(DATA_DIR/True_filename, index_col="Hours")
df_mod.index = df_mod.index.astype(int)
df_mod.index.name = "Hour"
df_mod.columns = df_mod.columns.astype(int)

Pred_hours_mod = [(start_hour + hh) % 24 for hh in range(len(df_mod.index))]
num_intervals = len(Pred_hours_mod)
# Convert your distance to kilometers
N = Dis.shape[0]
distance_km = Dis / 1000

In [ ]:
# Helper: get correct time-slice index from a departure time (hours since 0)
def slice_idx_from_departure(dep_time_h: float) -> int:
    # Which hour-of-day are we in? 0..23, aligned with Pred_hours start
    # dep_time_h is measured from "time 0" (your start), so wrap around 24
    hh = int(np.floor(dep_time_h / interval)) % num_intervals
    return hh

In [ ]:
# Compute cost matrices
distance_cost = (Dis / 1000) * C_o   # Euros

# --- run bidir‐Dijkstra on W ---
updated_full_cost  = np.zeros((num_locations, num_locations))
updated_base_dist  = np.zeros((num_locations, num_locations))
spaths_static = {}
edges_static = {}

# local weight function that closes over W
def weight_cost(u, v, data):
    return distance_cost[u, v]

for i in range(num_locations):
    for j in range(num_locations):
        if i == j:
            updated_full_cost[i, j] = 0.0
            updated_base_dist[i, j] = 0.0
            spaths_static[(i, j)]         = [i]
        else:
            cost, path = nx.bidirectional_dijkstra(G, i, j, weight=weight_cost)
            updated_full_cost[i, j] = cost
            spaths_static[(i, j)] = path
            edges = list(zip(path[:-1], path[1:]))
            edges_static[(i, j)] = edges
            # sum raw distances (meters) along path
            updated_base_dist[i, j] = sum(Dis[u, v] for u, v in edges)
            
distance_matrix_mod = updated_base_dist

### 2) MC loop to calculate outage probability 

In [ ]:
all_diffs = []   # store Difference_h values per run
all_true = []    # shape (MC, num_nodes)

for mc in range(MC):
    
    # Create random multipliers between 0.8 and 1.2, same shape as df_mod
    random_factors = np.random.normal(loc=MC_mean, scale=MC_std, size=df_mod.shape)     # np.random.uniform(MC_per_lb, MC_per_ub, size=df_mod.shape)
    #random_factors = np.random.uniform(MC_per_lb, MC_per_ub)
    df_randomized = df_mod * random_factors
    df_randomized.head()    # Keep same structure (Hours × IDs)

    speed_matrix_list_mod = []
    for h in Pred_hours_mod:
        M = np.zeros((N, N), dtype=float)
        speeds = df_randomized.loc[h]  # Series of speeds (km/h) by RoadID

        # dynamic edges
        for (i, j), Rid in dynamic_edges.items():
            w = float(speeds[Rid])
            M[i, j] = w; M[j, i] = w

        # static edges
        for (i, j), w in static_edges.items():
            w = float(w)
            M[i, j] = w; M[j, i] = w

        speed_matrix_list_mod.append(M)
        
    # Build travel-time matrices (hours) with modified speeds
    travel_time_matrix_list_mod = []
    for speed_mat in speed_matrix_list_mod:
        safe_speed = np.where(speed_mat > 0, speed_mat, 1e-6)  # avoid divide-by-zero
        tt = distance_km / safe_speed
        np.fill_diagonal(tt, 0.0)
        travel_time_matrix_list_mod.append(tt)

    # reuse precomputed paths; recompute times only
    updated_travel_time_list_mod = []
    for t in range(num_intervals):
        tt_mat = np.zeros((num_locations, num_locations))
        for i in range(num_locations):
            for j in range(num_locations):
                if i == j:
                    tt_mat[i, j] = 0.0
                    continue
                edges = edges_static[(i, j)]
                tt_mat[i, j] = sum(travel_time_matrix_list_mod[t][u, v] for u, v in edges)
        updated_travel_time_list_mod.append(tt_mat)
        
    #travel_time_matrix_list_mod = updated_travel_time_list_mod
    
    # Propagate along each route to compute new departure times l_new[i]
    # Convention: l_new[0] = 0 (depot). Each customer node gets a single l_new (first time we visit it).
    l_new = {i: None for i in V_range}
    l_new[0] = 0.0

    for route in routes:
        # route like [0, i1, i2, ..., 0]
        # start at depot
        current_dep = 0.0
        for idx in range(len(route) - 1):
            i = route[idx]
            j = route[idx + 1]

            # pick TT matrix slice based on departure from i
            h = slice_idx_from_departure(current_dep)
            tt_ij = float(updated_travel_time_list_mod[h][i, j])

            # arrive at j
            arrival_j = current_dep + tt_ij

            # set departure time at j (service except depot)
            if j == 0:
                # back to depot; no service time needed for next dep from depot in this route
                next_dep_j = arrival_j
            else:
                # if first time we reach j, record its departure time
                next_dep_j = arrival_j + service_time
                if l_new[j] is None:
                    l_new[j] = next_dep_j

            # move on
            current_dep = next_dep_j

    # Fill any still-None (shouldn't happen, but just in case)
    for i in V_range:
        if l_new[i] is None:
            l_new[i] = 0.0
            
    diffs = [l_new[i] - l_MILP[i] for i in V_range]
    trues = [l_new[i] for i in V_range]
    all_diffs.append(diffs)
    all_true.append(trues)

### 3) Calculate outage probability

In [ ]:
all_true_arr = np.asarray(all_true)   # shape (MC, num_nodes)

# Map node IDs to column indices
node_to_col = {node: k for k, node in enumerate(V_range)}
cols = [node_to_col[n] for n in range(0, N) if n in node_to_col]

# Check per run if any node in 0..10 exceeds B_bar
any_exceed_per_run = (all_true_arr[:, cols] > B_bar).any(axis=1)

# Outage probability
outage_prob = any_exceed_per_run.mean()

print(f"Outage probability (any node 0–10 exceeds B_bar): {outage_prob:.4f}")
print(f"Total runs with outage: {any_exceed_per_run.sum()} / {MC}")

### Node Performances

In [ ]:
time_unit_in_seconds = 3600  # <-- change if needed

# --- 1. Map nodes to columns and exclude node 0 ---

node_to_col = {node: k for k, node in enumerate(V_range)}
nodes_excl_0 = [node for node in V_range if node != 0]
cols_excl_0  = [node_to_col[n] for n in nodes_excl_0]

# True departure times for nodes != 0  → shape (MC, num_nodes-1)
true_excl_0 = all_true_arr[:, cols_excl_0]

# --- 2. Lateness for each node and each MC run (excluding node 0), in SECONDS ---

lateness_base_units = np.maximum(0.0, true_excl_0 - B_bar)  # same unit as input
lateness_sec = lateness_base_units * time_unit_in_seconds   # convert to seconds

# --- 3. For each MC run: how many nodes on time vs delayed (excluding node 0) ---

served_on_time_excl_0 = (true_excl_0 <= B_bar).sum(axis=1)   # shape (MC,)
served_late_excl_0    = (true_excl_0 >  B_bar).sum(axis=1)   # shape (MC,)

print("Average # of nodes on time per run (excluding node 0):",
      served_on_time_excl_0.mean())
print("Average # of nodes late per run (excluding node 0):   ",
      served_late_excl_0.mean())

## Step 6. Plot the results

In [ ]:
# Increase font sizes globally
plt.rcParams.update({
    "font.size": 14,
    "axes.titlesize": 16,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12
})

# --- 4+6. Combined figure: bar of #BSs late + lateness histogram (log counts) ---

# Unique counts and how often they occur (for late BSs)
unique_late, counts_late = np.unique(served_late_excl_0, return_counts=True)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Left subplot: BSs served with delay
axes[0].bar(unique_late, counts_late, width=0.8)
axes[0].set_title("BSs served with delay")
axes[0].set_xlabel("No. of BSs late")
axes[0].set_ylabel("No. of simulation runs")
axes[0].set_xticks(unique_late)
axes[0].grid(axis="y", alpha=0.3)

# Right subplot: Overall lateness histogram (log-scale counts)
axes[1].hist(lateness_sec.flatten(), bins=10, log=True)
axes[1].set_xlabel("Lateness (seconds)")
axes[1].set_ylabel("Frequency (log scale)")
axes[1].set_title("Overall lateness distribution")

plt.tight_layout()
plt.show()

### Average lateness calculations (in seconds)

In [ ]:
avg_lateness_per_bs_sec = lateness_sec.mean(axis=0)  # shape: (n_BS,)

print("Average lateness per BS (including zero-lateness runs), in seconds:")
for bs_id, avg_sec in zip(nodes_excl_0, avg_lateness_per_bs_sec):
    print(f"  BS {bs_id}: {avg_sec:.2f} s")
    
# --- Average lateness per BS, ignoring on-time runs (only positive lateness) ---

late_mask = lateness_sec > 0  # True where there is delay

# Sum of lateness over late runs only
sum_late_per_bs = (lateness_sec * late_mask).sum(axis=0)  # shape: (n_BS,)

# Number of late runs per BS
count_late_per_bs = late_mask.sum(axis=0)                 # shape: (n_BS,)

# Avoid division by zero using np.divide with 'where'
avg_lateness_pos_per_bs_sec = np.divide(
    sum_late_per_bs,
    count_late_per_bs,
    out=np.zeros_like(sum_late_per_bs, dtype=float),
    where=count_late_per_bs > 0
)

print("\nAverage lateness per BS (ignoring on-time runs), in seconds:")
for bs_id, avg_sec, n_late in zip(nodes_excl_0, avg_lateness_pos_per_bs_sec, count_late_per_bs):
    if n_late > 0:
        print(f"  BS {bs_id}: {avg_sec:.2f} s   (based on {n_late} late runs)")
    else:
        print(f"  BS {bs_id}: 0.00 s")

# Overall average lateness across all BSs and all runs (including zeros)
overall_avg_lateness_sec = lateness_sec.mean()
print(f"\nOverall average lateness (all BSs, all runs, including zeros): "
      f"{overall_avg_lateness_sec:.2f} s")

#  Average positive lateness (only when there *is* a delay)
positive_lateness = lateness_sec[lateness_sec > 0]

if positive_lateness.size > 0:
    overall_avg_positive_lateness_sec = positive_lateness.mean()
    print("Average lateness *conditional on being late* (only > 0), in seconds:",
          f"{overall_avg_positive_lateness_sec:.2f} s")
else:
    print("No positive lateness observed: all BSs are always on time.")

### Plot departure time distribution

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

# Boxplot of departure times for each BS across MC runs
ax.boxplot(
    [true_excl_0[:, j] for j in range(true_excl_0.shape[1])],
    labels=nodes_excl_0,
    showfliers=True
)

# Deadline line
ax.axhline(B_bar, linestyle="--", label="Deadline")

ax.set_xlabel("BS (node ID)")
ax.set_ylabel("Departure time (in hour)")
ax.set_title("Departure times for all BSs (excluding Depot)")
plt.xticks()
ax.legend()
plt.tight_layout()
plt.show()

### Plot Lateness distribution per node (excluding node 0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

ax.boxplot(
    [lateness_sec[:, j] for j in range(lateness_sec.shape[1])],
    labels=nodes_excl_0,
    showfliers=True
)

ax.set_xlabel("BS ID")
ax.set_ylabel("Lateness (seconds)")
ax.set_title("Lateness distribution per BS (excluding Depot)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Compare MILP vs. recomputed (modified speeds)
rows = []
for i in V_range:
    rows.append({
        "Node": i,
        "MILP_departure_h": round(l_MILP[i], 3),     # hours with 4 decimals
        "True_departure_h": round(l_new[i], 3),
        "Difference_h": round(l_new[i] - l_MILP[i], 3),
    })
cmp_df = pd.DataFrame(rows).sort_values("Node").reset_index(drop=True)

# Pretty print & optionally save
print("\nDeparture time comparison (hours):")
print(cmp_df.to_string(index=False))

### Average differences across MC runs

In [ ]:
all_diffs = np.array(all_diffs)  # shape (MC, num_nodes)
all_true = np.array(all_true)    # (MC, num_nodes)

avg_diffs = all_diffs.mean(axis=0)
avg_true  = all_true.mean(axis=0)

# Max values across MC runs
max_diffs = all_diffs.max(axis=0)
max_true  = all_true.max(axis=0)

# Build DataFrame
cmp_mc_df = pd.DataFrame({
    "Node": V_range,
    "MILP_departure_h": [round(l_MILP[i], 3) for i in V_range],
    "Avg_True_departure_h": np.round(avg_true, 3),
    "Avg_Difference_h": np.round(avg_diffs, 3),
    "Max_True_departure_h": np.round(max_true, 3),
    "Max_Difference_h": np.round(max_diffs, 3)
})

print("\nAverage results across MC runs:")
print(cmp_mc_df.to_string(index=False))

### Checking the results

In [ ]:
# check exact equality
equal_exact = np.array_equal(distance_matrix_, distance_matrix_mod)

# check numerical closeness (safer for floats)
equal_close = np.allclose(distance_matrix_, distance_matrix_mod, rtol=1e-9, atol=1e-12)

print("Exact equal:", equal_exact)
print("Numerically close:", equal_close)

# if not close, print maximum difference
if not equal_close:
    diff = np.abs(distance_matrix_ - distance_matrix_mod)
    print("Max difference:", diff.max())

In [ ]:
from pandas.testing import assert_frame_equal

df_m = pd.read_csv(DATA_DIR/MILP_filename)
df_t = pd.read_csv(DATA_DIR/True_filename)

try:
    assert_frame_equal(df_m.sort_index(axis=1).reset_index(drop=True),
                       df_t.sort_index(axis=1).reset_index(drop=True),
                       check_dtype=False, check_like=True)
    print("Inputs identical")
except AssertionError as e:
    print("Input mismatch:", e)

# Show where they diverge
cols = set(df_m.columns).intersection(df_t.columns)
for c in sorted(cols):
    ne = (df_m[c].reset_index(drop=True) != df_t[c].reset_index(drop=True))
    if getattr(df_m[c], "dtype", None) == float:
        ne = (df_m[c].reset_index(drop=True).round(6) !=
              df_t[c].reset_index(drop=True).round(6))
    if ne.any():
        i = ne.idxmax()
        print(f"First diff in {c} at row {i}: {df_m[c].iloc[i]} vs {df_t[c].iloc[i]}")
        break
    
# Example guards to neutralize alignment and rounding differences
X = df_m.to_numpy(copy=True)    # no index alignment
# compute Lb_departure_h_raw = ...
# compute True_departure_h_raw = ...
MILP_departure_h = pd.Series(cmp_df.MILP_departure_h).round(6).mod(24)
True_departure_h = pd.Series(cmp_df.True_departure_h).round(6).mod(24)

print("Same length:", len(MILP_departure_h) == len(True_departure_h))
print("Max abs diff:", (MILP_departure_h - True_departure_h).abs().max())